In [1]:
!pip install statsmodels

Defaulting to user installation because normal site-packages is not writeable


In [2]:
!pip install linearmodels

Defaulting to user installation because normal site-packages is not writeable


In [1]:
import numpy as np
import pandas as pd

from collections import Counter
import math
import scipy.stats as ss
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm
from statsmodels.discrete.discrete_model import Logit

from linearmodels import PanelOLS


pd.set_option('display.max_columns', None)

In [5]:
type_map = {
    'decision_date' : 'str',
    'filing_date' : 'str',
    'Court Name' : 'category',
    'Party of Appointing President' : 'category',
    'CIRCUIT' : 'category',
    'JURIS' : 'category',
    'NOS' : 'category',
    'ORIGIN' : 'category',
    'RESIDENC' : 'category',
    'CLASSACT' : 'category',
    'DEMANDED' : 'float64',
    'TERMDATE' : 'str',
    'DISP' : 'category',
    'PROCPROG' : 'category',
    'NOJ' : 'category',
    'AMTREC' : 'category',
    'JUDGMENT' : 'category',
    'TAPEYEAR' : 'float64',
    'district' : 'category',
    'office' : 'category',
    'county' : 'category',
    'TRCLACT' : 'category',
    'PROSE' : 'category',
    'arbit' : 'category',
    'transoff' : 'category',
    'trmarb' : 'category',
    'ifp' : 'category',
    'statuscd' : 'category'
}

cols_to_drop = [
    'decision_date',
#     'filing_date',
    'TERMDATE',
    'TAPEYEAR'
]

df = pd.read_csv(
    '/scratch/ayl316/ttml_mr_data/processed_data/cases.csv.zip', 
    dtype = type_map, 
    parse_dates = ['decision_date', 'filing_date', 'TERMDATE']
).rename(columns = {
    'Court Name' : 'court_name',
    'Party of Appointing President' : 'party'
}).drop(columns = cols_to_drop)


df['filing_year'] = pd.DatetimeIndex(df['filing_date']).year
# df['filing_year'] = df['filing_year'].astype(str).astype('category')
df['filing_year'] = df['filing_year'].fillna(0)
df['filing_year'] = df['filing_year'].astype(int)
df = df.drop(columns = ['filing_date'])

for col in ['party', 'TRCLACT', 'PROSE', 'arbit', 'transoff', 'trmarb', 'ifp', 'statuscd']:
    if not '-8' in df[col].cat.categories:
        df[col] = df[col].cat.add_categories('-8')
    df[col] = df[col].fillna('-8')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1219459 entries, 0 to 1219458
Data columns (total 25 columns):
 #   Column       Non-Null Count    Dtype   
---  ------       --------------    -----   
 0   court_name   1219459 non-null  category
 1   party        1219459 non-null  category
 2   CIRCUIT      1219459 non-null  category
 3   JURIS        1219459 non-null  category
 4   NOS          1219459 non-null  category
 5   ORIGIN       1219459 non-null  category
 6   RESIDENC     1219459 non-null  category
 7   CLASSACT     1219459 non-null  category
 8   DEMANDED     1219459 non-null  float64 
 9   DISP         1219459 non-null  category
 10  PROCPROG     1219459 non-null  category
 11  NOJ          1219459 non-null  category
 12  AMTREC       1219459 non-null  category
 13  JUDGMENT     1219459 non-null  category
 14  district     1219459 non-null  category
 15  office       1219459 non-null  category
 16  county       1219459 non-null  category
 17  TRCLACT      1219459 non-nu

In [33]:
ohe_cols = [
    'CLASSACT',
    'JURIS',
    'ORIGIN',
    'office',
    'NOS',
    'RESIDENC'
]

fe_cols = [
    'filing_year',
    'district'
]

target_col = 'party'

feature_cols = ohe_cols + fe_cols

In [34]:
df[feature_cols]

,CLASSACT,JURIS,ORIGIN,office,NOS,RESIDENC,filing_year,district
0,-8.0,2.0,1.0,8,510.0,-8.0,2016,16
1,-8.0,2.0,1.0,8,510.0,-8.0,2016,16
2,-8.0,2.0,1.0,8,510.0,-8.0,2016,16
3,-8.0,2.0,1.0,8,510.0,-8.0,2016,16
4,-8.0,2.0,1.0,8,510.0,-8.0,2016,16
...,...,...,...,...,...,...,...,...
1219454,-8.0,4.0,2.0,4,110.0,15.0,2016,65
1219455,-8.0,4.0,2.0,4,110.0,15.0,2016,65
1219456,-8.0,3.0,1.0,2,442.0,-8.0,2015,15
1219457,-8.0,3.0,1.0,2,442.0,-8.0,2015,15


In [35]:
df = df[(df[target_col] == 'Republican') | (df[target_col] == 'Democratic')]
df = df[df['filing_year'] != 0]

# df = df.sample(n = 50000)

for col in feature_cols:
    if df[col].dtype.name == 'category':
        df[col] = df[col].cat.remove_unused_categories()

X = df[ohe_cols]
y = df[target_col]


y = y.cat.add_categories(['1', '0'])
y[y == 'Democratic'] = '1'
y[y == 'Republican'] = '0'
y = y.cat.remove_unused_categories()

In [36]:
enc = OneHotEncoder(drop = 'first')
enc.fit(X)
X_ohe = enc.transform(X).toarray()

scaler = StandardScaler().fit(X_ohe)
X_scaled = pd.DataFrame(
    scaler.transform(X_ohe),
    columns = enc.get_feature_names(ohe_cols)
)

for fe_col in fe_cols:
    X_scaled[fe_col] = df[fe_col]

X_scaled['filing_year'] = pd.to_datetime(X_scaled['filing_year'], format='%Y')

X_scaled = X_scaled.set_index(fe_cols)
y = y.reindex(fe_cols)
# X_scaled.index = X_scaled.index.swaplevel(0,1)
# y.index = y.index.swaplevel(0, 1)
# X_scaled = sm.add_constant(X_scaled)

In [37]:
X_scaled

CLASSACT_-9.0  CLASSACT_0.0  CLASSACT_1.0  CLASSACT_2.0  \
filing_year district                                                            
2016-01-01  16            -0.003184     -0.096055     -0.107851     -0.012377   
            16            -0.003184     -0.096055     -0.107851     -0.012377   
            16            -0.003184     -0.096055     -0.107851     -0.012377   
            16            -0.003184     -0.096055     -0.107851     -0.012377   
            16            -0.003184     -0.096055     -0.107851     -0.012377   
...                             ...           ...           ...           ...   
2008-01-01  83            -0.003184     -0.096055     -0.107851     -0.012377   
            83            -0.003184     -0.096055     -0.107851     -0.012377   
            83            -0.003184     -0.096055     -0.107851     -0.012377   
            83            -0.003184     -0.096055     -0.107851     -0.012377   
            83            -0.003184     -0.096055     -0.107851     -0.012377   

                      CLASSACT_3.0  JURIS_2.0  JURIS_3.0  JURIS_4.0  \
filing_year district                                                  
2016-01-01  16           -0.003184   1.784443  -1.230558  -0.396759   
            16           -0.003184   1.784443  -1.230558  -0.396759   
            16           -0.003184   1.784443  -1.230558  -0.396759   
            16           -0.003184   1.784443  -1.230558  -0.396759   
            16           -0.003184   1.784443  -1.230558  -0.396759   
...                            ...        ...        ...        ...   
2008-01-01  83           -0.003184  -0.560399  -1.230558   2.520424   
            83           -0.003184  -0.560399  -1.230558   2.520424   
            83           -0.003184  -0.560399   0.812639  -0.396759   
            83           -0.003184  -0.560399   0.812639  -0.396759   
            83           -0.003184  -0.560399   0.812639  -0.396759   

                      JURIS_5.0  ORIGIN_1.0  ORIGIN_2.0  ORIGIN_3.0  \
filing_year district                                                  
2016-01-01  16        -0.007198    0.440500   -0.344437   -0.057251   
            16        -0.007198    0.440500   -0.344437   -0.057251   
            16        -0.007198    0.440500   -0.344437   -0.057251   
            16        -0.007198    0.440500   -0.344437   -0.057251   
            16        -0.007198    0.440500   -0.344437   -0.057251   
...                         ...         ...         ...         ...   
2008-01-01  83        -0.007198   -2.270147    2.903290   -0.057251   
            83        -0.007198   -2.270147    2.903290   -0.057251   
            83        -0.007198    0.440500   -0.344437   -0.057251   
            83        -0.007198    0.440500   -0.344437   -0.057251   
            83        -0.007198    0.440500   -0.344437   -0.057251   

                      ORIGIN_4.0  ORIGIN_5.0  ORIGIN_6.0  ORIGIN_7.0  \
filing_year district                                                   
2016-01-01  16         -0.178255   -0.141843   -0.035948   -0.006959   
            16         -0.178255   -0.141843   -0.035948   -0.006959   
            16         -0.178255   -0.141843   -0.035948   -0.006959   
            16         -0.178255   -0.141843   -0.035948   -0.006959   
            16         -0.178255   -0.141843   -0.035948   -0.006959   
...                          ...         ...         ...         ...   
2008-01-01  83         -0.178255   -0.141843   -0.035948   -0.006959   
            83         -0.178255   -0.141843   -0.035948   -0.006959   
            83         -0.178255   -0.141843   -0.035948   -0.006959   
            83         -0.178255   -0.141843   -0.035948   -0.006959   
            83         -0.178255   -0.141843   -0.035948   -0.006959   

                      ORIGIN_8.0  ORIGIN_9.0  office_1  office_2  office_3  \
filing_year district                                                         
2016-01-01  16         -0.0

In [38]:
FE = PanelOLS(
    X_scaled, 
    y,
    entity_effects = True,
    time_effects = True
)

result = FE.fit(
    cov_type = 'clustered',
     cluster_entity = True
)

ValueError: The index on the time dimension must be either numeric or date-like

In [13]:


data = pd.read_csv(r"https://raw.githubusercontent.com/LOST-STATS/LOST-STATS.github.io/master/Model_Estimation/Data/Fixed_Effects_in_Linear_Regression/Scorecard.csv")

# Set the index for fixed effects
data = data.set_index(['inst_name', 'year'])



In [14]:
data

,,unitid,state_abbr,pred_degree_awarded_ipeds,earnings_med,count_not_working,count_working,repay_rate
inst_name,year,,,,,,,
Alabama A & M University,2007,100654,AL,3,36600.0,116.0,1139.0,NaN
University of Alabama at Birmingham,2007,100663,AL,3,40800.0,366.0,2636.0,NaN
Amridge University,2007,100690,AL,3,NaN,6.0,25.0,NaN
University of Alabama in Huntsville,2007,100706,AL,3,49300.0,122.0,975.0,NaN
Alabama State University,2007,100724,AL,3,30500.0,210.0,1577.0,NaN
...,...,...,...,...,...,...,...,...
Pima Medical Institute-Dillon,2016,489812,MT,1,NaN,NaN,NaN,0.375000
Arthur's Beauty College-Jonesboro,2016,489830,AR,1,NaN,NaN,NaN,NaN
Palm Beach Academy of Health & Beauty-Distinct Health Educational Center,2016,489900,FL,1,NaN,NaN,NaN,0.578431
